# Scraping et formatage des données DATAN

<div style="text-align: right"> André Mounier </div>

---
Application sur les données DATAN

In [20]:
import os
import pandas as pd
import tqdm
from pathlib import Path
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from textwrap import wrap
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.gridspec import GridSpec
from bs4 import BeautifulSoup
from lxml import etree
import requests

### Liste des votes

In [2]:
df_allvotes = pd.read_html('https://datan.fr/votes/legislature-16')[0]
df_allvotes = df_allvotes.set_index('N°')

In [3]:
df_allvotes.iloc[3702]

Date                                               27-10-2022
Titre       L'amendement n° 1882 de M. Schreck à l'article...
Résultat                                               REJETÉ
Name: 403, dtype: object

### Tests de récupération des détails des votants (groupes et parlementaires)

In [4]:
vote_number = 403

In [5]:
vote_infos = pd.read_html('https://datan.fr/votes/legislature-16/vote_{}'.format(vote_number))
df_desc, _, df_groups, df_deputees = vote_infos[:4]
# df_deputees

### Création d'un unique tableau de données

In [6]:
output_path = os.path.join('output','DATAN')
Path(output_path).mkdir(parents=True, exist_ok=True)

In [7]:
dict_group = {'ni':'Non inscrit (NI)',
              'dem':'Démocrate (MoDem et Indépendants) (DEM)',
              'lfi-nupes':'La France insoumise - NUPES (LFI-NUPES)',
              'soc-a':'Socialistes et apparentés - NUPES (SOC)',
              'gdr-nupes':'Gauche démocrate et républicaine - NUPES (GDR-NUPES)',
              'lr':'Les Républicains (LR)',
              'hor':'Horizons et apparentés (HOR)',
              'rn':'Rassemblement National (RN)',
              'ecolo':'Écologiste - NUPES (ECOLO)',
              'liot':'Libertés, Indépendants, Outre-mer et Territoires (LIOT)',
              're':'Renaissance (RE)'}

dict_group_deputees = {'ni':7,
                       'dem':50,
                       'lfi-nupes':75,
                       'soc-a':31,
                       'gdr-nupes':22,
                       'lr':61,
                       'hor':31,
                       'rn':88,
                       'ecolo':21,
                       'liot':22,
                       're':169}

dict_group_2024 = {'ni':None,
                   'dem':'EN',
                   'lfi-nupes':'FP',
                   'soc-a':'FP',
                   'gdr-nupes':'FP',
                   'lr':'RN',
                   'hor':'EN',
                   'rn':'RN',
                   'ecolo':'FP',
                   'liot':None,
                   're':'EN'}

In [8]:
if 'datan.csv' not in os.listdir(output_path):

    # initilisation des colonnes du tableau de sortie
    columns = {'vote':[],'date':[],'type':[],'dossier':[]}
    for group in dict_group.keys():
        columns['{}_nb-pour'.format(group)] = []
        columns['{}_nb-contre'.format(group)] = []
        columns['{}_nb-abstention'.format(group)] = []
        columns['{}_participation'.format(group)] = []
    
    # construction du tableau
    for vote in tqdm.tqdm(df_allvotes.index):
        vote_infos = pd.read_html('https://datan.fr/votes/legislature-16/vote_{}'.format(vote))
        df_desc, _, df_groups, df_deputees = vote_infos[:4]
        
        # formatage du tableau des votes de groupe
        df_groups_formatted = df_groups.set_index('Groupe')
        
        # formatage du tableau descriptif du vote
        df_desc_formatted = df_desc.T
        df_desc_formatted = df_desc_formatted.rename(columns={i:n for i,n in enumerate(df_desc_formatted.iloc[1])})
        df_desc_formatted
        
        # ajout de la date
        try:
            date = df_desc_formatted['Date'].values[-1]
        except KeyError:
            date = np.nan
        
        # ajout du type de vote 
        try:
            type_vote = df_desc_formatted['Type de vote'].values[-1]
        except KeyError:
            type_vote = np.nan
            
        # ajout du dossier
        try:
            dossier = df_desc_formatted['Dossier'].values[-1]
        except KeyError:
            dossier = np.nan
            
        columns['vote'].append(vote)
        columns['date'].append(date)
        columns['type'].append(type_vote)
        columns['dossier'].append(dossier)
        
        # ajout des statistiques de votes par groupe parlementaire
        for group, group_name in dict_group.items():
            filter_group = df_groups_formatted.index.str.startswith(group_name[:10])
            df_group = df_groups_formatted.loc[filter_group]
            nb_pour, nb_contre, nb_abstention, ratio_particip = df_group[['Pour','Contre','Abstention','Participation']].values[0]
            ratio_particip = float(ratio_particip.replace('%',''))/100
            
            columns['{}_nb-pour'.format(group)].append(nb_pour)
            columns['{}_nb-contre'.format(group)].append(nb_contre)
            columns['{}_nb-abstention'.format(group)].append(nb_abstention)
            columns['{}_participation'.format(group)].append(ratio_particip)

    # sauvegarde
    datan = pd.DataFrame().from_dict(columns)
    datan.to_csv(os.path.join(output_path,'datan.csv'),index=False)

### Ouverture et manipulation du jeu de données

In [9]:
datan = pd.read_csv(os.path.join(output_path,'datan.csv'))

In [10]:
# pas terminé
dict_categories = {'Vieillesse et fin de vie':['Accompagnement des malades et fin de vie',
                                              ],
                   'Loi de finances':['Projet de loi de financement de la sécurité sociale pour 2024',
                                      'Projet de loi de finances de fin de gestion pour 2023',
                                      'Projet de loi de finances pour 2024',
                                      'Programmation des finances publiques pour les années 2023 à 2027',
                                      'Projet de loi de financement rectificative de la sécurité sociale pour 2023',
                                      'Financement de la sécurité sociale pour 2023',
                                      'Projet de loi de finances rectificative pour 2022',
                                      'Projet de loi de finances pour 2023',
                                      'Loi de finances rectificative pour 2022',
                                      "Approbation des comptes de la sécurité sociale pour l'année 2022"],
                   'Dématérialisation':["Proposition de loi visant à poursuivre la dématérialisation de l'état civil du ministère de l'Europe et des affaires étrangères",],
                   'Économie des entreprises':['Accroître le financement des entreprises et l’attractivité de la France',],
                   'Fiscalité':['Charge fiscale de la pension alimentaire',
                                'Assurer une justice patrimoniale au sein de la famille',],
                   'Retraite':['Calculer la retraite de base des non-salariés agricoles en fonction de leurs seules vingt-cinq meilleures années de revenus'],
                   'Union européenne':['Projet de loi portant diverses dispositions d’adaptation au droit de l’Union européenne en matière d’économie, de finances, de transition écologique, de droit pénal, de droit social et en matière agricole',
                                      ],
                   'Commerce international':["Accord commercial entre l'Union européenne et le Mercosur",
                                             'Accords entre la France et le Sénégal du 7 septembre 2021 et entre la France et le Sri Lanka du 23 février 2022'],
                   'Nucléaire':['Projet de loi relatif à l’organisation de la gouvernance de la sûreté nucléaire et de la radioprotection pour répondre au défi de la relance de la filière nucléaire',],
                   'Outre-mer':['Projet de loi ratifiant l’ordonnance n° 2023-285 du 19 avril 2023 portant extension et adaptation à la Polynésie française, à la Nouvelle-Calédonie et aux îles Wallis et Futuna de diverses dispositions législatives relatives à la santé',
                                "Projet de loi ratifiant l'ordonnance n° 2023-389 du 24 mai 2023 modifiant les dispositions du code général de la propriété des personnes publiques relatives à la Polynésie française",
                                'Projet de loi ratifiant les ordonnances relatives à la partie législative du livre VII du code monétaire et financier et portant diverses dispositions relatives à l’outre-mer',
                                'Projet de loi ratifiant l’ordonnance n° 2021-1605 du 8 décembre 2021 étendant et adaptant à la fonction publique des communes de Polynésie française certaines dispositions statutaires relatives à la fonction publique territoriale'],
                   'Service public':["Professionnaliser l'enseignement de la danse en tenant compte de la diversité des pratiques",
                                     'Pérenniser les jardins d’enfants gérés par une collectivité publique ou bénéficiant de financements publics',],
                   'Santé':['Le dérapage du coût pour l’État de la couverture santé des étrangers en situation irrégulière et des demandeurs d’asile provenant de pays d’origine sûrs et au nombre d’étrangers en situation irrégulière',
                            "Mieux manger en soutenant les Français face à l'inflation et en favorisant l'accès à une alimentation saine",
                            "Abrogation de l'obligation vaccinale contre la covid-19 dans les secteurs médicaux, paramédicaux et d'aide à la personne et visant à la réintégration des professionnels et étudiants suspendus",
                            'Amélioration de l’accès aux soins par la confiance aux professionnels de santé',
                            "Améliorer l'accès aux soins par l'engagement territorial des professionnels",
                            "Améliorer l'accès aux soins par la territorialisation et la formation",],
                   'Finance':["Élargir l'assiette de la taxe sur les transactions financières",
                              'Compléter les dispositions applicables au Haut Conseil de stabilité financière'],
                   'Éducation':['Faciliter la mobilité internationale des alternants pour un « Erasmus de l’apprentissage »'],
                   'Droit':['Adapter le droit de la responsabilité civile aux enjeux actuels',
                            'Allonger la durée de l’ordonnance de protection et à créer l’ordonnance provisoire de protection immédiate',
                            "Améliorant l'efficacité des dispositifs de saisie et de confiscation des avoirs criminels",
                            'Confidentialité des consultations des juristes d’entreprise'],
                   'Écologie':['Appeler à un accord ambitieux lors de la quinzième conférence des parties à la convention sur la diversité biologique',
                               "Approbation du premier amendement et du protocole à la convention d'Espoo"],
                   'Culture':["Assurer la pérennité des établissements de spectacles cinématographiques et l'accès au cinéma dans les outre-mer"],
                   'Fraude fiscale':['Approbation des conventions entre la France et le Danemark et entre la France et la Grèce pour l’élimination de la double imposition en matière d’impôts sur le revenu et la prévention de l’évasion et de la fraude fiscales'],
                   'Sécurité sociale':['Constitutionnaliser la sécurité sociale'],
                  }


datan_dossiers = set(datan.dossier.values)
for l in dict_categories.values():
    for dossier in l:
        datan_dossiers.remove(dossier)


In [11]:
datan_nodossier = datan[[type(e)==float for e in datan.dossier]]
datan_nodossier

,vote,date,type,dossier,ni_nb-pour,ni_nb-contre,ni_nb-abstention,ni_participation,dem_nb-pour,dem_nb-contre,...,ecolo_nb-abstention,ecolo_participation,liot_nb-pour,liot_nb-contre,liot_nb-abstention,liot_participation,re_nb-pour,re_nb-contre,re_nb-abstention,re_participation
57,4048,05 juin 2024,NaN,NaN,0,0,1,0.14,26,0,...,0,0.14,0,1,0,0.05,50,0,0,0.30
84,4021,03 juin 2024,Motion de censure,NaN,2,0,0,0.29,0,0,...,0,0.00,0,0,0,0.00,0,0,0,0.00
85,4020,03 juin 2024,Motion de censure,NaN,2,0,0,0.29,0,0,...,0,0.52,0,0,0,0.00,0,0,0,0.00
113,3992,30 mai 2024,Proposition de résolution,NaN,1,0,0,0.14,0,0,...,0,0.38,2,0,0,0.09,1,0,8,0.05
114,3991,30 mai 2024,Proposition de résolution,NaN,2,0,0,0.29,1,1,...,0,0.71,4,0,0,0.18,1,1,45,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3875,230,06 octobre 2022,Article,NaN,1,0,0,0.25,21,0,...,0,0.04,1,0,0,0.05,22,0,0,0.13
3876,229,06 octobre 2022,Article,NaN,1,0,0,0.25,21,0,...,0,0.04,1,0,0,0.05,24,1,0,0.15
3916,189,04 août 2022,Article,NaN,0,3,0,0.33,30,0,...,0,0.43,0,1,2,0.19,94,0,0,0.55
3921,184,02 août 2022,NaN,NaN,4,0,0,0.44,37,0,...,0,0.48,5,0,0,0.31,92,0,0,0.53


### Étude des similitudes RN/ENS

In [12]:
def make_space_above(fig,axes, topmargin=1):
    """ increase figure size to make topmargin (in inches) space for 
        titles, without changing the axes sizes"""
    s = fig.subplotpars
    w, h = fig.get_size_inches()
    figh = h - (1-s.top)*h  + topmargin
    fig.subplots_adjust(bottom=s.bottom*h/figh, top=1-topmargin/figh)
    fig.set_figheight(figh)

In [13]:
# définition du chemin de sauvegarde
fig_path = os.path.join(output_path,'figs')
Path(fig_path).mkdir(parents=True, exist_ok=True)

# définition des couleurs utilisées (provenant du logo NFP)
fpcolor = tuple(e/255 for e in (193, 34, 72))
fpcolor_dark = tuple(e/255 for e in (146, 34, 92))

for dossier in tqdm.tqdm(list(set(datan.dossier.values))):
    # exclusion des votes sans dossier
    if type(dossier)!=str:
        continue
        
    df = datan[datan.dossier==dossier]
    
    # exclusion des amendements 
    df = df[~df.type.isin(['Amendement','Sous-amendement'])]

    # moyenne des positions de chaque sous-groupe pour les différents votes du dossier
    kept_columns = ['{}_nb-pour'.format(group) for group in dict_group.keys()]
    kept_columns += ['{}_nb-contre'.format(group) for group in dict_group.keys()]
    kept_columns += ['{}_nb-abstention'.format(group) for group in dict_group.keys()]
    kept_columns += ['{}_participation'.format(group) for group in dict_group.keys()]
    df = df[kept_columns]
    df = df.mean()

    # initialisation des votes par groupes
    dict_group_votes = {'EN':{'pour':0,'contre':0,'abstention':0},
                        'FP':{'pour':0,'contre':0,'abstention':0},
                        'RN':{'pour':0,'contre':0,'abstention':0},}
    
    for group in dict_group_2024.keys():
        pour = df['{}_nb-pour'.format(group)]
        contre = df['{}_nb-contre'.format(group)]
        abstention = df['{}_nb-abstention'.format(group)]
        
        group_2024 = dict_group_2024.get(group)

        # exclusion des sous-groupes hors NFP, EN, RN
        if group_2024 is None:
            continue
            
        dict_group_votes[group_2024]['pour'] += pour
        dict_group_votes[group_2024]['contre'] += contre
        dict_group_votes[group_2024]['abstention'] += abstention

    # position du groupe définie comme le maximum de voix entre pour, contre ou abstention
    dict_group_results = {g:max(dict_group_votes[g], key=dict_group_votes[g].get) for g in dict_group_votes.keys()}

    ###############################################################################################################
    #fig, (ax1,ax2) = plt.subplots(1,2,figsize=(6,3))
    fig = plt.figure(figsize=(6,5))

    gs = GridSpec(2,2, figure=fig, height_ratios=[1,0.33], hspace=0.05, wspace=0.05)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])

    # définition des positions verticales et des tailles des logos
    ref_size = 50
    dict_group_graphics = {'EN':(0.33,ref_size/822),
                           'FP':(0.66,ref_size/296),
                           'RN':(0.05,ref_size/281),}

    # titre du dossier sur plusieurs lignes
    title_dossier = wrap(dossier,width=50)
    nb_lines_title = len(title_dossier)
    title_dossier = '\n'.join(title_dossier)
    fig.suptitle(title_dossier,y=0.98, color=fpcolor_dark, weight='bold')

    # modification de l'espace entre le titre et les graphes pour et contre
    base_margin = 0.3
    top_margin = base_margin+(1-base_margin)/4*nb_lines_title
    make_space_above(fig,(ax1,ax2), topmargin=top_margin)   

    # ajout des titres des boites pour et contre, et abstention
    ax1.text(0.5,0.978,'POUR',horizontalalignment='center', backgroundcolor='w', color=fpcolor, weight='bold')
    ax2.text(0.5,0.978,'CONTRE',horizontalalignment='center', backgroundcolor='w', color=fpcolor, weight='bold')
    ax3.text(0.1,0.5,'ABST.',horizontalalignment='left', verticalalignment='center', backgroundcolor='w', color='grey', weight='bold')
    
    # suppression des ticks et labels mais conservation des contours
    ax1.get_xaxis().set_ticks([])
    ax1.get_yaxis().set_ticks([])
    ax2.get_xaxis().set_ticks([])
    ax2.get_yaxis().set_ticks([])
    ax3.axis(False)
    
    # changement de couleur des contours
    for ax in [ax1, ax2]:
        plt.setp(ax.spines.values(), color=fpcolor)
        plt.setp([ax.get_xticklines(), ax.get_yticklines()], color=fpcolor)

    # ajout des logos aux bonnes tailles et dans les bonnes boites
    for g,v in dict_group_results.items():
        if v == 'pour':
            (height, zoom), logo_path = dict_group_graphics.get(g), os.path.join('input','logos','logo_{}.png'.format(g))
            im = OffsetImage(plt.imread(logo_path), zoom=zoom)
            ab = AnnotationBbox(im, (0.5, height), xycoords='axes fraction', frameon=False, box_alignment=(0.5,0))
            ax1.add_artist(ab)
        elif v == 'contre':
            (height, zoom), logo_path = dict_group_graphics.get(g), os.path.join('input','logos','logo_{}.png'.format(g))
            im = OffsetImage(plt.imread(logo_path), zoom=zoom)
            ab = AnnotationBbox(im, (0.5, height), xycoords='axes fraction', frameon=False, box_alignment=(0.5,0))
            ax2.add_artist(ab)
        else:
            (_, zoom), logo_path = dict_group_graphics.get(g), os.path.join('input','logos','logo_{}.png'.format(g))
            im = OffsetImage(plt.imread(logo_path), zoom=zoom)
            ab = AnnotationBbox(im, (0.5, 0.5), xycoords='axes fraction', frameon=False, box_alignment=(0.5,0.5))
            ax3.add_artist(ab)
        

    # enregistrement de l'image
    plt.savefig(os.path.join(fig_path,'{}.png'.format(dossier.replace(' ','_')[:90])),bbox_inches='tight')
    plt.close()
    

100%|█████████████████████████████████████████| 165/165 [00:15<00:00, 10.88it/s]


### Liste des votes des députés RN

In [14]:
if 'datan_RN_deputees.csv' not in os.listdir(output_path):

    # initilisation des colonnes du tableau de sortie
    columns = {'vote':[],'date':[],'type':[],'dossier':[],'depute':[],'issue':[]}
    
    # construction du tableau
    for vote in tqdm.tqdm(df_allvotes.index):
        vote_infos = pd.read_html('https://datan.fr/votes/legislature-16/vote_{}'.format(vote))
        df_desc, _, df_groups, df_deputees = vote_infos[:4]
        
        # formatage du tableau des votes de groupe
        df_deputees_formatted = df_deputees[df_deputees.Groupe=='Rassemblement National']
        
        # formatage du tableau descriptif du vote
        df_desc_formatted = df_desc.T
        df_desc_formatted = df_desc_formatted.rename(columns={i:n for i,n in enumerate(df_desc_formatted.iloc[1])})
        df_desc_formatted
        
        # ajout de la date
        try:
            date = df_desc_formatted['Date'].values[-1]
        except KeyError:
            date = np.nan
        
        # ajout du type de vote 
        try:
            type_vote = df_desc_formatted['Type de vote'].values[-1]
        except KeyError:
            type_vote = np.nan
            
        # ajout du dossier
        try:
            dossier = df_desc_formatted['Dossier'].values[-1]
        except KeyError:
            dossier = np.nan
        
        for dep,vote_issue in zip(df_deputees_formatted.Député, df_deputees_formatted.Vote):
            columns['vote'].append(vote)
            columns['date'].append(date)
            columns['type'].append(type_vote)
            columns['dossier'].append(dossier)
            columns['depute'].append(dep)
            columns['issue'].append(vote_issue)

    # sauvegarde
    datan = pd.DataFrame().from_dict(columns)
    datan.to_csv(os.path.join(output_path,'datan_RN_deputees.csv'),index=False)

### Récupération des informations d'un vote en particulier (titre, description) à partir de son numéro

In [35]:
def get_vote_infos(vote_id):
    # scraping du site datan.fr
    url = 'https://datan.fr/votes/legislature-16/vote_{}'.format(vote_id)

    # lecture du code html
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}) 
    html = requests.get(url, headers=HEADERS) 
    soup = BeautifulSoup(html.content, 'lxml')
    dom = etree.HTML(str(soup)) 
    
    # xpath correspondant au titre du vote
    vote_name = dom.xpath('/html/body/main/div[3]/div[1]/div/div/div[1]/h1')[0].text
    return vote_name

In [31]:
datan_RN = pd.read_csv(os.path.join(output_path,'datan_RN_deputees.csv'))

In [38]:
if 'vote_desc' not in datan_RN.columns:
    list_vote_ids = list(set(datan_RN.vote.values))
    dict_vote_ids_name = {vid:get_vote_infos(vid) for vid in tqdm.tqdm(list_vote_ids)}
    
    datan_RN['vote_desc'] = [dict_vote_ids_name.get(vid) for vid in datan_RN.vote.values]
    datan_RN.to_csv(os.path.join(output_path,'datan_RN_deputees.csv'),index=False)

### Ajout d'information sur les députés en exercice

Données issues de : https://data.assemblee-nationale.fr/acteurs/deputes-en-exercice

In [45]:
deputees_AN = pd.read_csv(os.path.join('input','liste_deputes.csv'))
deputees_DATAN = pd.read_csv(os.path.join('output','RN_deputees_information.csv'))
datan_RN_votes = pd.read_csv(os.path.join(output_path,'datan_RN_deputees.csv'))

In [64]:
# ajout de l'ID de chaque député
dict_name_to_id = {k:v for k,v in zip(deputees_DATAN.Name,deputees_DATAN['Deputy ID'])}
datan_RN_votes['depute_id'] = [dict_name_to_id.get(name) for name in datan_RN_votes.depute]

# ajout du département et de la circonscription
dict_id_to_dep = {k:s[s.find("(")+1:s.find(")")] for k,s in zip(deputees_DATAN['Deputy ID'],deputees_DATAN['Constituency'])}
dict_id_to_circo = {k:int(e) for k,e in zip(deputees_AN.identifiant,deputees_AN['Numéro de circonscription'])}
datan_RN_votes['depute_departement'] = [dict_id_to_dep.get(depute_id) for depute_id in datan_RN_votes.depute_id]
for depute_id in datan_RN_votes.depute_id:
    try:
        int(dict_id_to_circo.get(depute_id))
    except: 
        print(depute_id)
datan_RN_votes['depute_circonscription'] = [int(dict_id_to_circo.get(depute_id)) for depute_id in datan_RN_votes.depute_id]

796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
796114
794286
794286
794286
794286
794286
794286
794286
794286
794286
794286

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [62]:
datan_RN_votes

,vote,date,type,dossier,depute,issue,vote_desc,depute_id,depute_dep,depute_departement,depute_circonscription
0,4105,07 juin 2024,Amendement,Accompagnement des malades et fin de vie,Christophe Bentz,CONTRE,L'amendement n° 518 de Mme Iborra à l'article ...,794434,52,52,1.0
1,4105,07 juin 2024,Amendement,Accompagnement des malades et fin de vie,Jocelyn Dessigny,CONTRE,L'amendement n° 518 de Mme Iborra à l'article ...,793166,02,02,5.0
2,4105,07 juin 2024,Amendement,Accompagnement des malades et fin de vie,Edwige Diaz,CONTRE,L'amendement n° 518 de Mme Iborra à l'article ...,793928,33,33,11.0
3,4105,07 juin 2024,Amendement,Accompagnement des malades et fin de vie,Thierry Frappé,CONTRE,L'amendement n° 518 de Mme Iborra à l'article ...,794678,62,62,10.0
4,4105,07 juin 2024,Amendement,Accompagnement des malades et fin de vie,Marine Hamelet,CONTRE,L'amendement n° 518 de Mme Iborra à l'article ...,795900,82,82,2.0
...,...,...,...,...,...,...,...,...,...,...,...
115590,2,11 juillet 2022,Motion de rejet préalable,Maintien provisoire d'un dispositif de veille ...,Emeric Salmon,POUR,"La motion de rejet préalable, déposée par Mme ...",794954,70,70,2.0
115591,2,11 juillet 2022,Motion de rejet préalable,Maintien provisoire d'un dispositif de veille ...,Emmanuel Taché de la Pagerie,POUR,"La motion de rejet préalable, déposée par Mme ...",793382,13,13,16.0
115592,2,11 juillet 2022,Motion de rejet préalable,Maintien provisoire d'un dispositif de veille ...,Jean-Philippe Tanguy,POUR,"La motion de rejet préalable, déposée par Mme ...",795778,80,80,4.0
115593,2,11 juillet 2022,Motion de rejet préalable,Maintien provisoire d'un dispositif de veille ...,Lionel Tivoli,POUR,"La motion de rejet préalable, déposée par Mme ...",793298,06,06,2.0
